In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     


── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


[1] "2b-data"

 # Overfitting and Regularization

In the realm of predictive modeling, the pursuit of creating a model that perfectly fits the training data can inadvertently lead to a phenomenon known as overfitting. Overfitting occurs when a model becomes excessively complex, capturing not only the genuine patterns within the data but also the noise and random fluctuations present in the training set. 

The hyper-adaptation to the training data renders the model less capable of generalizing to new, unseen data, as it effectively memorizes the training examples rather than discerning meaningful relationships. As a result, an overfitted model may exhibit impressive performance on the training data but performs poorly when faced with real-world scenarios. The delicate balance between capturing essential patterns and avoiding the trap of overfitting underscores the importance of techniques like regularization, which aim to ensure model generalization by restraining excessive complexity.

There are three Regularization techniques we will be dealing with, all of which use the idea of penalizing terms to tackle overfitting.

But before we go any further, let's have a look at the data.

**Data Dictionary**

* Product_ID:- Unique identifier for each electronic device. <br>
* Product_Weight:- Weight of the device  <br>
* Energy_Efficiency:- Energy efficiency rating (e.g., "Energy Efficient", "Standard"). <br>
* Visibility_Index:- The % of the total display area of all products in a store allocated to the particular product <br>
* Product_Category:- Category of the electronic device (e.g., "Laptops", "Smartphones"). <br>
* Product_Price:- Price of the device in dollars<br>
* Store_ID:- Unique identifier for the store.<br>
* Store_Established_Year:- Number of years since the store was established.<br>
* Store_Size:- Size classification of the store (e.g., "Small", "Medium").<br>
* Location_Type:- The type of city in which the store is located<br>
* Store_Type:- Type of store (e.g., "Electronics Store Type1").<br>
* Sales_Performance:- Sales of the electronic device in the particular store. This is the outcome variable to be predicted.


## Visualizing the data

Let's visualize this all in the form of a Data Frame

In [2]:
data <- read.csv("/kaggle/input/2b-data/2b-data.csv")
head(data)


,Product_ID,Product_Weight,Energy_Efficiency,Visibility_Index,Product_Category,Product_Price,Store_ID,Store_Established_Year,Store_Size,Location_Type,Store_Type,Sales_Performance
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>
1,FDA15,9.300,Energy Efficient,0.01604730,Laptops,249.8092,OUT049,14,Medium,Tier 1,Electronics Store Type1,3735.1380
2,DRC01,5.920,Standard,0.01927822,Smartphones,48.2692,OUT018,4,Medium,Tier 3,Electronics Store Type2,443.4228
3,FDN15,17.500,Energy Efficient,0.01676007,Tablets,141.6180,OUT049,14,Medium,Tier 1,Electronics Store Type1,2097.2700
4,FDX07,19.200,Standard,0.06613203,Monitors,182.0950,OUT010,15,Small,Tier 3,Retail Electronics Store,732.3800
5,NCD19,8.930,Energy Efficient,0.06613203,Refrigerators,53.8614,OUT013,26,High,Tier 3,Electronics Store Type1,994.7052
6,FDP36,10.395,Standard,0.06613203,Microwaves,51.4008,OUT018,4,Medium,Tier 3,Electronics Store Type2,556.6088


## Ridge Regression
Ridge regression is a linear regression technique that incorporates L2 regularization to address issues in predictive modeling (overfitting, multi-colinearity etc).
Linear regression, aims to minimize the sum of squared residuals whereas Ridge regression introduces a penalty term proportional to the square of the magnitude of the coefficients. This penalty, controlled by a hyperparameter (often denoted as lambda), discourages large coefficient values, effectively constraining the model’s complexity, enhancing its stability and generalization performance.


**1)** Perform Ridge Regression on the training data and compare the predictions with the test data to check for the fit of the model. (Hint: Use the glmnet library) (2 marks) <br>
You can split the dataset into 70% train and 30% test.  

In [3]:
##Write your code here
if (!requireNamespace("glmnet", quietly = TRUE)) {
    install.packages("glmnet")
}
library(glmnet)


Loading required package: Matrix




Attaching package: ‘Matrix’




The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack




Loaded glmnet 4.1-8



In [4]:
train<-data[c(-1,-7)]
Y<-train[c(10)]
X <- model.matrix(Sales_Performance~., train)[, -1]



In [5]:
set.seed(567)
part <- sample(2, nrow(X), replace = TRUE, prob = c(0.7, 0.3))
X_train<- X[part == 1,]
X_cv<- X[part == 2,]

Y_train<- Y[part == 1,]
Y_cv<- Y[part == 2,]

In [6]:
ridge_reg <- glmnet(X_train, Y_train, alpha = 0)

**2)** Is it possible for you to somehow conduct hyperparameter tuning and find the best lambda value for the Ridge Regression model? (Hint: use the cv.glmnet function)   (1 mark)

In [7]:
# Find the best lambda via cross-validation
ridge_reg1 <- cv.glmnet(X_train, Y_train, alpha = 0)

bestlam <- ridge_reg1$lambda.min
print(bestlam)

[1] 98.68757


In [8]:
# Predict using the best lambda
ridge.pred <- predict(ridge_reg, s = bestlam, newx = X_cv)

**3)** With the optimal lambda, build the model again and print the coefficients of the various independent variables. What can you comment about the relationship between lambda and the strength of regularization?        (2 marks)


In [9]:
# Rebuild the Ridge Regression model using the optimal lambda
ridge_final <- glmnet(X_train, Y_train, alpha = 0, lambda = bestlam)

# Extract and print the coefficients of the Ridge Regression model
ridge_coef <- coef(ridge_final)
print("Coefficients of the Ridge Regression Model:")
print(ridge_coef)


[1] "Coefficients of the Ridge Regression Model:"


28 x 1 sparse Matrix of class "dgCMatrix"
                                                    s0
(Intercept)                                 189.995530
Product_Weight                               -2.020927
Energy_EfficiencyStandard                    39.549175
Visibility_Index                           -619.146188
Product_CategoryCameras                      34.695765
Product_CategoryHeadphones                   80.149754
Product_CategoryHome Theater Systems        118.185029
Product_CategoryKeyboards                    45.863053
Product_CategoryLaptops                      50.553944
Product_CategoryMicrowaves                   52.866816
Product_CategoryMiscellaneous Electronics   -56.398372
Product_CategoryMonitors                     47.216039
Product_CategoryMouse                        89.121489
Product_CategoryPrinters                     82.709920
Product_CategoryRefrigerators                -3.091872
Product_CategorySmartphones                 -23.841540
Product_CategorySpeaker


Small Lambda: Minimal regularization; coefficients closer to those of ordinary least squares; risk of overfitting.
Large Lambda: Strong regularization; coefficients shrink towards zero; reduces overfitting but may lead to underfitting if too large.

## Lasso Regression

Lasso regression is similar to Ridge Regression except that instead of L2 regularization, it employs L1 regularization to address the very same issues that Ridge Regression addresses. 

There are however, a couple of differences between the two. The first and most obvious being that since Lasso Regression implements L1 regularization, the penalty term in this case is proportional to the absolute value of the coefficient. 

Another point to note is that unlike its Ridge counterpart, Lasso Regression can push some coefficients to exactly 0. This effectively drops the feature from the predictive model (Similar to how we drop values through visual analysis). Lasso Regression can thus be used effectively for Feature Selection as well.

**1)** Write code to build a Lasso Regression model similar to how you built the Ridge Regression model. This time incorporate hyperparameter tuning right away. So first print the optimal lambda value.                  (2 marks)

In [10]:
# Define the lambda grid
lambda <- 10^seq(10, -2, length = 100)
# Fit the Lasso model
lasso_reg1 <- cv.glmnet(X_train, Y_train, alpha = 1, lambda = lambda)

# Print the optimal lambda value
bestlam <- lasso_reg1$lambda.min
print(bestlam)


[1] 18.73817


**2)** Display the coefficients of all the variables. Do you notice some variables being dropped out? Which ones are they?   (1 mark)

In [11]:
# Display the coefficients of all the variables
lasso_reg <- glmnet(X_train, Y_train, alpha = 1, lambda = bestlam)
lasso.coef <- predict(lasso_reg, type = "coefficients", s = bestlam)
print(lasso.coef)


28 x 1 sparse Matrix of class "dgCMatrix"
                                                    s1
(Intercept)                                 122.217413
Product_Weight                                .       
Energy_EfficiencyStandard                    18.522185
Visibility_Index                            -71.233399
Product_CategoryCameras                       .       
Product_CategoryHeadphones                    .       
Product_CategoryHome Theater Systems          .       
Product_CategoryKeyboards                     .       
Product_CategoryLaptops                       .       
Product_CategoryMicrowaves                    .       
Product_CategoryMiscellaneous Electronics     .       
Product_CategoryMonitors                      .       
Product_CategoryMouse                         .       
Product_CategoryPrinters                      .       
Product_CategoryRefrigerators                 .       
Product_CategorySmartphones                   .       
Product_CategorySpeaker

As we can see 20 variables are dropped.      (Note: Your dropped variables and answers for best lambda might be completely different depending on how the model is built)

## Elastic Net Regression

Elastic Net regression, an advanced form of linear regression, combines the benefits of L1 (Lasso) and L2 (Ridge) regularization methods. By integrating both penalty terms, Elastic Net overcomes the limitations of each, offering resilience against multicollinearity, aiding feature selection, and preventing overfitting. 

This approach makes Elastic Net a very versatile approach for achieving accurate and efficient models by finding a middle ground between dropping parameters and retaining important predictors. 

**1)** Build your Elastic Net Regression model incorporating all the steps we previously followed for ridge and lasso regression. (Play around with the alpha value and find out how it affects the model)            (2 marks)


Identify and list the variables that have been dropped in the Elastic Net model. How does this compare to the variables dropped in the Lasso model? 

In [12]:
## Write your code here
#If we choose alpha as 0.3:
lambda <- 10^seq(10, -2, length = 100)

# Train the Elastic Net model with alpha = 0.3
elastic_net_model <- cv.glmnet(X_train, Y_train, alpha = 0.3, lambda = lambda)

# Get the best lambda from cross-validation
best_lambda <- elastic_net_model$lambda.min
best_coef <- coef(elastic_net_model, s = best_lambda)
print(best_coef)

28 x 1 sparse Matrix of class "dgCMatrix"
                                                    s1
(Intercept)                                 140.806479
Product_Weight                                .       
Energy_EfficiencyStandard                    30.348990
Visibility_Index                           -236.076233
Product_CategoryCameras                       .       
Product_CategoryHeadphones                    9.843827
Product_CategoryHome Theater Systems          .       
Product_CategoryKeyboards                     .       
Product_CategoryLaptops                       .       
Product_CategoryMicrowaves                    .       
Product_CategoryMiscellaneous Electronics    -3.195723
Product_CategoryMonitors                      .       
Product_CategoryMouse                         .       
Product_CategoryPrinters                      .       
Product_CategoryRefrigerators                -2.075726
Product_CategorySmartphones                   .       
Product_CategorySpeaker

We can see that fewer variables are dropped than in Lasso.

In [13]:
#If we iterate for a range of alpha values and choose the model that has minimum MSE:
data <- read.csv("/kaggle/input/2b-data/2b-data.csv")
train<-data[c(-1,-7)]
Y<-train[c(10)]
X <- model.matrix(Sales_Performance~., train)[, -1]
set.seed(567)
part <- sample(2, nrow(X), replace = TRUE, prob = c(0.7, 0.3))
X_train<- X[part == 1,]
X_cv<- X[part == 2,]

Y_train<- Y[part == 1,]
Y_cv<- Y[part == 2,]

lambda <- 10^seq(10, -2, length = 100)
alpha_values <- seq(0, 1, length = 11)  # Exploring a range of alpha values

# Initialize a list to store results
results <- list()
best_model <- NULL
best_mse <- Inf  # Start with a high MSE to find the minimum

# Perform Elastic Net Regression for different alpha values
for (alpha in alpha_values) {
  

  # Fit Elastic Net model
  elastic_net_model <- cv.glmnet(X_train, Y_train, alpha = alpha, lambda = lambda)
  bestlam <- elastic_net_model$lambda.min
  
  
  elastic_net_pred <- predict(elastic_net_model, s = bestlam, newx = X_cv)
  mse <- mean((Y_cv - elastic_net_pred)^2)
  
  # Store the best model
  if (mse < best_mse) {
    best_mse <- mse
    best_model <- elastic_net_model
    best_alpha <- alpha
    best_lambda <- bestlam
  }
}

# Display the results for the best model
cat("Best Elastic Net Model - Alpha:", best_alpha, " - Lambda:", best_lambda, " - MSE:", best_mse, "\n")

# Extract and display the coefficients of the best model
best_coef <- coef(best_model, s = best_lambda)
print(best_coef)

Best Elastic Net Model - Alpha: 0.8  - Lambda: 18.73817  - MSE: 1273661 


28 x 1 sparse Matrix of class "dgCMatrix"
                                                   s1
(Intercept)                                 117.94479
Product_Weight                                .      
Energy_EfficiencyStandard                    26.50738
Visibility_Index                           -142.69033
Product_CategoryCameras                       .      
Product_CategoryHeadphones                    2.97123
Product_CategoryHome Theater Systems          .      
Product_CategoryKeyboards                     .      
Product_CategoryLaptops                       .      
Product_CategoryMicrowaves                    .      
Product_CategoryMiscellaneous Electronics     .      
Product_CategoryMonitors                      .      
Product_CategoryMouse                         .      
Product_CategoryPrinters                      .      
Product_CategoryRefrigerators                 .      
Product_CategorySmartphones                   .      
Product_CategorySpeakers                

19 variables are dropped,not much lesser than Lasso mostly because best alpha is 0.8 which is very close to alpha value=1 of Lasso. Various product categories and store attributes, have coefficients of zero, effectively being excluded. This is similar to Lasso's property of feature selection, where less relevant features are set to zero.

What does that tell you about the number of hyperparameters in Elastic Net Regression compared to the other two models?

Elastic Net Regression combines features of both Ridge and Lasso Regression, leading to some unique behaviors in its coefficients:

Combining Regularization:<br>
Ridge Effect (L2 Norm): It shrinks coefficients towards zero but doesn’t make them exactly zero.<br>
Lasso Effect (L1 Norm): It can shrink coefficients to zero, effectively performing variable selection by excluding some predictors from the model.<br>
Impact on Coefficients:<br>
Balance of Shrinkage and Sparsity: Depending on the alpha parameter, Elastic Net applies a mix of Ridge and Lasso penalties. This can result in some coefficients being exactly zero (like Lasso) and others being shrunk but not zero (like Ridge).<br>
Handling Correlated Features: Elastic Net is particularly useful when dealing with correlated features, as it can select groups of related features rather than picking just one.<br>

Hyperparameters in Elastic Net Regression
Elastic Net involves two key hyperparameters:

Lambda (Regularization Strength): Controls the overall intensity of the regularization applied to the model.<br>
Alpha (Mixing Parameter): Determines the balance between Lasso and Ridge penalties.
alpha = 0: The model behaves like Ridge Regression.
alpha = 1: The model behaves like Lasso Regression.
0 < alpha < 1: The model incorporates a blend of both penalties.

